<a href="https://colab.research.google.com/github/bhavya-kashyap/AMC_crosswalk/blob/main/abstract_pubmed_webscrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install bs4
!pip install pymed

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [98.9 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hi

In [2]:
import pandas as pd
import numpy as np
from pymed import PubMed
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import math
from google.colab import files

In [ ]:
# Data parameters
# https://www.webofscience.com/wos/woscc/summary/f05a3dd0-ac52-4364-859d-aea68cf4f4d3-583fd5ef/relevance/1
# SO = (MMWR RECOMMENDATIONS AND REPORTS OR NEW ENGLAND JOURNAL OF MEDICINE OR NATURE MEDICINE OR NATURE REVIEWS IMMUNOLOGY OR MORBIDITY AND MORTALITY WEEKLY REPORT OR LANCET OR PHYSIOLOGICAL REVIEWS OR NATURE REVIEWS DRUG DISCOVERY OR ANNUAL REVIEW OF PATHOLOGY: MECHANISMS OF DISEASE OR JOURNAL OF CLINICAL ONCOLOGY OR LANCET GLOBAL HEALTH OR ANNALS OF ONCOLOGY OR ANNUAL REVIEW OF PUBLIC HEALTH OR NATURE REVIEWS DISEASE PRIMERS OR ACCOUNTS OF CHEMICAL RESEARCH OR ANNUAL REVIEW OF CLINICAL PSYCHOLOGY OR JOURNAL OF EXPERIMENTAL MEDICINE OR MOLECULAR SYSTEMS BIOLOGY OR ANNUAL REVIEW OF MICROBIOLOGY OR JAMA - JOURNAL OF THE AMERICAN MEDICAL ASSOCIATION OR LANCET DIGITAL HEALTH OR REPORTS ON PROGRESS IN PHYSICS OR NATURE BIOMEDICAL ENGINEERING OR EMBO JOURNAL OR JAMA PSYCHIATRY OR NATURE REVIEWS MOLECULAR CELL BIOLOGY OR CELL OR NATURE REVIEWS GENETICS OR NATURE BIOTECHNOLOGY OR NATURE REVIEWS CANCER OR NATURE GENETICS OR ANNUAL REVIEW OF BIOCHEMISTRY OR NATURE CATALYSIS OR NATURE METHODS OR CANCER CELL OR MOLECULAR CELL OR NATURE CELL BIOLOGY OR LANCET DIABETES AND ENDOCRINOLOGY OR NATURE MICROBIOLOGY OR ANNUAL REVIEW OF PLANT BIOLOGY OR ANNUAL REVIEW OF GENETICS OR CELL HOST AND MICROBE OR GENOME BIOLOGY OR CELL METABOLISM OR NATURE REVIEWS ENDOCRINOLOGY OR NATURE STRUCTURAL AND MOLECULAR BIOLOGY OR NUCLEIC ACIDS RESEARCH OR PHARMACOLOGICAL REVIEWS OR ANNUAL REVIEW OF PHARMACOLOGY AND TOXICOLOGY OR NANO TODAY OR DRUG RESISTANCE UPDATES OR JOURNAL FOR IMMUNOTHERAPY OF CANCER OR TRENDS IN PHARMACOLOGICAL SCIENCES OR EMERGING MICROBES AND INFECTIONS OR PHARMACOLOGY AND THERAPEUTICS OR MOLECULAR THERAPY OR NPJ VACCINES OR ADVANCED DRUG DELIVERY REVIEWS OR NATURAL PRODUCT REPORTS OR CELL CHEMICAL BIOLOGY OR PROTEIN AND CELL OR CELLULAR AND MOLECULAR LIFE SCIENCES OR MEDICINAL RESEARCH REVIEWS OR NEUROPSYCHOPHARMACOLOGY OR ANTIVIRAL RESEARCH OR ACTA PHARMACEUTICA SINICA B OR PAIN OR THERANOSTICS OR ADVANCED HEALTHCARE MATERIALS OR BRITISH JOURNAL OF PHARMACOLOGY OR ENVIRONMENTAL POLLUTION OR ALZHEIMER'S AND DEMENTIA OR HEALTH AFFAIRS OR LANCET PLANETARY HEALTH OR IMPLEMENTATION SCIENCE OR INFECTIOUS DISEASE MODELLING OR BMJ GLOBAL HEALTH OR GLOBALIZATION AND HEALTH OR JOURNAL OF HEALTH ECONOMICS OR JOURNAL OF THE AMERICAN MEDICAL DIRECTORS ASSOCIATION OR MILBANK QUARTERLY OR EUROPEAN HEART JOURNAL. QUALITY OF CARE &AMP, CLINICAL OUTCOMES OR BMJ QUALITY AND SAFETY OR HEALTH SERVICES RESEARCH OR MEDICAL CARE RESEARCH AND REVIEW OR VALUE IN HEALTH OR POLICY AND INTERNET OR INTERNATIONAL JOURNAL OF DRUG POLICY OR INTERNATIONAL JOURNAL FOR EQUITY IN HEALTH OR JOURNAL OF GLOBAL HEALTH OR PHARMACOECONOMICS OR HEALTH RESEARCH POLICY AND SYSTEMS OR GLOBAL HEALTH RESEARCH AND POLICY OR INTERNATIONAL JOURNAL OF TRANSGENDER HEALTH OR JCO ONCOLOGY PRACTICE OR PREVENTING CHRONIC DISEASE OR NATURE OR SCIENCE OR SCIENCE ADVANCES OR PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES OF THE UNITED STATES OF AMERICA OR NATIONAL SCIENCE REVIEW OR SCIENCE BULLETIN OR RESEARCH OR JOURNAL OF ADVANCED RESEARCH OR ISCIENCE OR SCIENTIFIC REPORTS OR TSINGHUA SCIENCE AND TECHNOLOGY OR PROCEEDINGS OF THE JAPAN ACADEMY SERIES B: PHYSICAL AND BIOLOGICAL SCIENCES OR ADVANCED THEORY AND SIMULATIONS OR PROGRESS IN NATURAL SCIENCE: MATERIALS INTERNATIONAL OR PLOS ONE OR SCIENCE AND TECHNOLOGY STUDIES OR ROYAL SOCIETY OPEN SCIENCE OR RESEARCH IN SCIENCE AND TECHNOLOGICAL EDUCATION OR INTERNATIONAL JOURNAL OF BIFURCATION AND CHAOS IN APPLIED SCIENCES AND ENGINEERING OR LATIN AMERICAN RESEARCH REVIEW OR FRACTALS OR FACETS OR HELIYON OR JOURNAL OF KING SAUD UNIVERSITY - SCIENCE OR ARABIAN JOURNAL FOR SCIENCE AND ENGINEERING OR CLINICAL AND TRANSLATIONAL IMMUNOLOGY OR JOURNAL OF PAIN AND SYMPTOM MANAGEMENT OR INTERNATIONAL JOURNAL OF NURSING STUDIES OR NURSE EDUCATION TODAY OR JOURNAL OF NURSING SCHOLARSHIP OR JOURNAL OF PALLIATIVE MEDICINE OR PATIENT OR CLINICAL SIMULATION IN NURSING OR INTERNATIONAL NURSING REVIEW OR JOURNAL OF CLINICAL NURSING OR WORLDVIEWS ON EVIDENCE-BASED NURSING OR NURSE EDUCATION IN PRACTICE OR JOURNAL OF ADVANCED NURSING OR JOURNAL OF SCHOOL NURSING OR JOURNAL OF RELIGION AND HEALTH OR INTERNATIONAL JOURNAL OF NURSING SCIENCES OR NURSING OUTLOOK OR PALLIATIVE AND SUPPORTIVE CARE OR SYMBOLIC INTERACTION OR JOURNAL OF PROFESSIONAL NURSING OR JOURNAL OF TRANSCULTURAL NURSING OR JOURNAL OF NURSING EDUCATION OR BMC NURSING OR JOURNAL OF MULTIDISCIPLINARY HEALTHCARE)
# Publication years: last 5 years (2018 - 2022)
# Affiliations: Emory university
# Size of data: 1000 records
# primary focus is to get the pubmed ID

In [ ]:
df = pd.read_csv("Georgia_pubmed.csv", encoding='cp1252')
df.shape

(855, 11)

In [ ]:
df.head()
# df.drop(columns = ['Publication Type','Authors','Author Full Names', 'Group Authors','Source Title','Document Type', 'Publication Date','Publication Year', 'Issue','Start Page','End Page', 'Article Number','DOI','Volume', 'DOI Link', 'Early Access Date', 'UT (Unique WOS ID)', 'Web of Science Record'], inplace=True)

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI
0,23746772,Maternal and child undernutrition and overweig...,"Black RE, Victora CG, Walker SP, Bhutta ZA, Ch...",Lancet. 2013 Aug 3;382(9890):427-451. doi: 10....,Black RE,Lancet,2013,11-06-2013,NaN,NaN,10.1016/S0140-6736(13)60937-X
1,20448178,A draft sequence of the Neandertal genome,"Green RE, Krause J, Briggs AW, Maricic T, Sten...",Science. 2010 May 7;328(5979):710-722. doi: 10...,Green RE,Science,2010,08-05-2010,PMC5100745,NIHMS827403,10.1126/science.1188021
2,20716669,"Glutamate receptor ion channels: structure, re...","Traynelis SF, Wollmuth LP, McBain CJ, Menniti ...",Pharmacol Rev. 2010 Sep;62(3):405-96. doi: 10....,Traynelis SF,Pharmacol Rev,2010,19-08-2010,PMC2964903,NaN,10.1124/pr.109.002451
3,24352235,The complete genome sequence of a Neanderthal ...,"Prüfer K, Racimo F, Patterson N, Jay F, Sankar...",Nature. 2014 Jan 2;505(7481):43-9. doi: 10.103...,Prüfer K,Nature,2014,20-12-2013,PMC4031459,NIHMS541101,10.1038/nature12886
4,16382236,Restoring function in exhausted CD8 T cells du...,"Barber DL, Wherry EJ, Masopust D, Zhu B, Allis...",Nature. 2006 Feb 9;439(7077):682-7. doi: 10.10...,Barber DL,Nature,2006,31-12-2005,NaN,NaN,10.1038/nature04444


In [ ]:
baseurl = 'https://pubmed.ncbi.nlm.nih.gov/'
urls = []
for i in range(len(df)):
    urls.append(baseurl + str(df.loc[i]['PMID'].astype(int)) + '/')
df['URL'] = urls
df.head(10)

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI,URL
0,23746772,Maternal and child undernutrition and overweig...,"Black RE, Victora CG, Walker SP, Bhutta ZA, Ch...",Lancet. 2013 Aug 3;382(9890):427-451. doi: 10....,Black RE,Lancet,2013,11-06-2013,NaN,NaN,10.1016/S0140-6736(13)60937-X,https://pubmed.ncbi.nlm.nih.gov/23746772/
1,20448178,A draft sequence of the Neandertal genome,"Green RE, Krause J, Briggs AW, Maricic T, Sten...",Science. 2010 May 7;328(5979):710-722. doi: 10...,Green RE,Science,2010,08-05-2010,PMC5100745,NIHMS827403,10.1126/science.1188021,https://pubmed.ncbi.nlm.nih.gov/20448178/
2,20716669,"Glutamate receptor ion channels: structure, re...","Traynelis SF, Wollmuth LP, McBain CJ, Menniti ...",Pharmacol Rev. 2010 Sep;62(3):405-96. doi: 10....,Traynelis SF,Pharmacol Rev,2010,19-08-2010,PMC2964903,NaN,10.1124/pr.109.002451,https://pubmed.ncbi.nlm.nih.gov/20716669/
3,24352235,The complete genome sequence of a Neanderthal ...,"Prüfer K, Racimo F, Patterson N, Jay F, Sankar...",Nature. 2014 Jan 2;505(7481):43-9. doi: 10.103...,Prüfer K,Nature,2014,20-12-2013,PMC4031459,NIHMS541101,10.1038/nature12886,https://pubmed.ncbi.nlm.nih.gov/24352235/
4,16382236,Restoring function in exhausted CD8 T cells du...,"Barber DL, Wherry EJ, Masopust D, Zhu B, Allis...",Nature. 2006 Feb 9;439(7077):682-7. doi: 10.10...,Barber DL,Nature,2006,31-12-2005,NaN,NaN,10.1038/nature04444,https://pubmed.ncbi.nlm.nih.gov/16382236/
5,24168957,Aggressive medical treatment with or without s...,"Derdeyn CP, Chimowitz MI, Lynn MJ, Fiorella D,...",Lancet. 2014 Jan 25;383(9914):333-41. doi: 10....,Derdeyn CP,Lancet,2014,31-10-2013,PMC3971471,NIHMS560585,10.1016/S0140-6736(13)62038-3,https://pubmed.ncbi.nlm.nih.gov/24168957/
6,20203013,Metabolic syndrome and altered gut microbiota ...,"Vijay-Kumar M, Aitken JD, Carvalho FA, Cullend...",Science. 2010 Apr 9;328(5975):228-31. doi: 10....,Vijay-Kumar M,Science,2010,06-03-2010,PMC4714868,NIHMS312925,10.1126/science.1179721,https://pubmed.ncbi.nlm.nih.gov/20203013/
7,24196715,Attention to eyes is present but in decline in...,"Jones W, Klin A.",Nature. 2013 Dec 19;504(7480):427-31. doi: 10....,Jones W,Nature,2013,08-11-2013,PMC4035120,NIHMS527415,10.1038/nature12715,https://pubmed.ncbi.nlm.nih.gov/24196715/
8,22945416,A randomized controlled trial of the tumor nec...,"Raison CL, Rutherford RE, Woolwine BJ, Shuo C,...",JAMA Psychiatry. 2013 Jan;70(1):31-41. doi: 10...,Raison CL,JAMA Psychiatry,2013,05-09-2012,PMC4015348,NIHMS570243,10.1001/2013.jamapsychiatry.4,https://pubmed.ncbi.nlm.nih.gov/22945416/
9,16458769,Smallpox,"Moore ZS, Seward JF, Lane JM.",Lancet. 2006 Feb 4;367(9508):425-35. doi: 10.1...,Moore ZS,Lancet,2006,07-02-2006,NaN,NaN,10.1016/S0140-6736(06)68143-9,https://pubmed.ncbi.nlm.nih.gov/16458769/


In [ ]:
sample = df.head(20)

In [3]:
def driversetup():
      options = webdriver.ChromeOptions()
      #run Selenium in headless mode
      options.add_argument('--headless')
      options.add_argument('--no-sandbox')
      options.add_argument("enable-automation")
      #overcome limited resource problems
      options.add_argument('--disable-dev-shm-usage')
      options.add_argument("lang=en")
      #open Browser in maximized mode
      options.add_argument("start-maximized")
      #disable infobars
      options.add_argument("disable-infobars")
      #disable extension
      options.add_argument("--disable-browser-side-navigation")
      options.add_argument("--disable-gpu")
      options.add_argument("--disable-extensions")
      options.add_argument("--incognito")
      options.add_argument("--disable-blink-features=AutomationControlled")
      driver = webdriver.Chrome(options=options)
      driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")
      return driver

def pagesource(url, driver):
      driver = driver
      driver.get(url)
      time.sleep(5)
      soup = BeautifulSoup(driver.page_source)
      return soup

In [ ]:
d = driversetup()
abstracts = []
for i in range(len(df)):
  s = pagesource(df.loc[i]['URL'], d)
  abstract = list(s.findAll('div', class_="abstract-content selected"))
  a = ''
  for j in range(len(abstract)):
    a = a + ' ' + abstract[j].text
  abstracts.append(a)
  print(i, a)

df["PubMedAbstracts"] = abstracts

df.to_csv('GeorgiaPubmedAbstracts.csv', index=False)
files.download("GeorgiaPubmedAbstracts.csv")
abs = pd.DataFrame({'Abstracts' : abstracts})
abs.to_csv('GeorgiaAbstracts.csv')
files.download("GeorgiaAbstracts.csv")

In [ ]:
a = pd.read_csv('GeorgiaPubmedAbstracts.csv')
a.columns.values

array(['Unnamed: 0', 'PMID', 'Title', 'Authors', 'Citation',
       'First Author', 'Journal/Book', 'Publication Year', 'Create Date',
       'PMCID', 'NIHMS ID', 'DOI', 'URL', 'PubMedAbstracts'], dtype=object)

In [ ]:
# Pymed

pubmed = PubMed(tool="MyTool", email="bkkashyapbhavya97@gmail.com")
uni = ['Emory University', 'Morehouse School of Medicine', 'Mercer University', 'Augusta University', 'Pacific Northwest University of Health Sciences College of Osteopathic Medicine']
date = '("2005/01/01"[Date - Publication] : "2014/01/01"[Date - Publication])'
j = pd.
query = """(((MMWR RECOMMENDATIONS AND REPORTS"[JOURNAL]) OR (NEW ENGLAND JOURNAL OF MEDICINE"[JOURNAL]))"""
results = list(pubmed.query(query, max_results=25000))

In [ ]:
article_dict = {}
article_dict_list = []
als_articles = pd.DataFrame()

# Loop over the retrieved articles
for i, article in enumerate(results):
    article_dict = {}
    
    if not article.abstract:
        pass 
    
    else:
        article_dict['abstract'] = article.abstract
        
        try:
            article_dict["conclusions"]= article.conclusions
        except:
            article_dict["conclusions"]= 'none'
            
        article_dict["copyrights"] = article.copyrights
        article_dict["doi"] = article.doi
        
        try:
            article_dict["journal"] = article.journal
        except:
            article_dict["journal"] ='none'
            
        try:
            article_dict["methods"] = article.methods
        except:
            article_dict["methods"]='none'
        article_dict["publication_date"] = article.publication_date
        article_dict["pubmed_id"]= article.pubmed_id
        
        try:
            article_dict["results"]=article.results
        except:
            article_dict["results"]='none'
        article_dict["title"] = article.title

        try:
            article_dict["keywords"] = '; '.join(article.keywords) 
        except:
            article_dict["keywords"] = "none"

        authors = []
        affils = []
        for a,author in enumerate(article.authors):
            auth = []
            auth.append(author['lastname'])
            auth.append(author['firstname'])
            auth.append(author['initials'])
            author_name = ', '.join(str(x) for x in auth)
            authors.append(author_name)

            try:
                if author['affiliation']!= None:
                    affils.append(author['affiliation'])
                else:
                    affils.append('none listed')
            except:
                affils.append('none listed')


        article_dict["authors"] = '; '.join(authors) 
        article_dict["affiliations"] = '; '.join(affils) 


        article_dict_list.append(article_dict)
        #print(article.toJSON())
    
als_articles = pd.concat([pd.DataFrame(article_dict_list)])

als_articles.head()
len(als_articles)


0

In [ ]:
als_articles[['abstract', 'journal', 'title', 'affiliations']].head()
#als_articles.columns.values

,abstract,journal,title,affiliations
0,Early results of the Stenting and Aggressive M...,"Lancet (London, England)",Aggressive medical treatment with or without s...,Mallinckrodt Institute of Radiology and the De...
1,Maternal and child malnutrition in low-income ...,"Lancet (London, England)",Maternal and child undernutrition and overweig...,"Johns Hopkins University, Bloomberg School of ..."
2,Present mechanical devices are unable to achie...,"Lancet (London, England)",Trevo versus Merci retrievers for thrombectomy...,"Department of Neurology, Grady Memorial Hospit..."
3,Men who have sex with men (MSM) have been subs...,"Lancet (London, England)",Successes and challenges of HIV prevention in ...,"Rollins School of Public Health, Emory Univers..."
4,Pathogens specifically target both the caspase...,Nature reviews. Immunology,Viral infection and the evolution of caspase 8...,"Department of Microbiology and Immunology, Emo..."


In [ ]:
# https://www.kaggle.com/code/summerkrankin/pubmed-download-als/notebook